# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# load city list from weatherpy challenge
# Print df results
file = pd.read_csv("city_data.csv")
file.dropna()
file.head()

,City ID,Date,Country,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloud Cover
0,0,2020-04-09 13:46:05,US,Hilo,19.73,-155.09,71.60,78,5.06,1
1,1,2020-04-09 13:48:33,CA,Aklavik,68.22,-135.01,-2.20,84,3.36,75
2,2,2020-04-09 13:46:06,ZA,Cape Town,-33.93,18.42,82.40,14,11.41,100
3,3,2020-04-09 13:48:33,CV,São Filipe,14.90,-24.50,80.08,53,14.81,0
4,4,2020-04-09 13:48:33,LY,Benghazi,32.12,20.07,61.05,53,13.38,0


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Set Gmaps with the G-key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' from DF into lists
locations = file[["Lat", "Lng"]]
humidity = file["Humidity"].astype(float)

In [4]:
# Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Add heat layer over map 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Set up what my ideal weather would be
nice_temp=file.loc[(file['Max Temp']>=60)&(file['Max Temp']<=85)]
nice_wind=nice_temp.loc[(nice_temp['Wind Speed']<=5)]
nice_weather=nice_wind.loc[(nice_wind['Cloud Cover']==0)]
nice_weather.head()

,City ID,Date,Country,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloud Cover
67,134,2020-04-09 13:48:39,CN,Jingdezhen,29.29,117.21,63.05,61,3.22,0
229,296,2020-04-09 13:49:43,CN,Chaozhou,23.67,116.64,66.20,63,4.47,0
264,331,2020-04-09 13:49:46,AU,Broken Hill,-31.95,141.43,60.80,59,3.36,0
298,365,2020-04-09 13:49:50,AR,Concepción del Uruguay,-32.48,-58.24,64.00,64,1.99,0
335,402,2020-04-09 13:49:53,UA,Poroshkovo,48.67,22.75,73.40,17,2.24,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# create hotel file
hotel_df = nice_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# parameters
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}


In [11]:
# Use the stored locations from city_data to identify hotels
for index, row in hotel_df.iterrows():
    
    # latitudes and longitudes from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAAKdT7XSxHRWw2gl-ZD2VUl10lQftSw2-_bzVuCVwxYqUGV9ppJt-vtCVWPf5nvNOyhIb4wfR_2ZViB8vWh-XJytu18S8CUV4AfVqpAJGyPhbbGc-KzZ7mDCQ8-wUAit4HQWDkPgpkpj39kBC8snIDcRSdpQhEqhwy2W4DinPVUwKHSaxg-Yd3oy1wTp6qTXmTENd4r8M7McI_LC7tii0k1QrayaQ0OqmOWc8ZFNmNkkZ9WgGBJKzqdaGjBfpGvYvS3Dnq7SAJcgwlFllt2z3Ok_wE5MtGTyw0sCnolrOa3ZO_BXSru1eNYXt8KsnUARTNzraAvv99T-tQpYwPJKD8w-W7tchT-uPT_g9JAaMaj6t-nHsYMg4sgWSzCQGH6t2zcyTJl4FUVo1dkXe1YdKmAYSEPI4YxrDbpEGUbbDygOoXMsaFE9DWPSsDHIgoJ93Xr5nAiSyflcl",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 29.316957,
                    "lng": 117.22715
                },
                "viewport": {
                    "northeast": {
                        "lat": 29.31777082989272,
                        "lng": 117.2283681798927
                    },
                    "southwest": {
                        "lat": 29.31507117010728,
                        "

{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAAC43drfkm17df7duOngi874lVhoH1ZHtzg5vK-WeysXaHOJf7abpa_ZJ0QZSQ1-lC5iMEodpNpIq9Vvn3reChPxKLwRGbRVqGHqEGDY2ts11Z4JkYEeNRPbj6Y12JWmzXSNdMfM1yPjX9QFfjOLZbItD-TgaO6unawkzVAp2sYAamwmbhccJwDeRb9qZ-WFUo5SDTI-JvP-Mr2Dkom-qqSbsGWr-S9xhW26UYRCK2wVsIQR4P91mb8WXjCVkfEY1aqI6yl-rBnutn7PXjKUfZrf34d1X7cw-u7QFvSY6L-hybNfy8jFgrKND-Ukr9HAlFMF5qiSfEwNrCXZGaz_BtUbHZOl8-ykKT4JwKJC3L2Xu7a4FI1cn4iD7Wsc8J9dT61ubrVqg9anzYCrsYA65948SEBegZsjnwlHwXAXm-t4HHVAaFEE1n2FTouR2A_EnURVgaTPA8k_P",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 23.365841,
                    "lng": 116.704035
                },
                "viewport": {
                    "northeast": {
                        "lat": 23.36725037989272,
                        "lng": 116.7051766798927
                    },
                    "southwest": {
                        "lat": 23.36455072010727,
                        

{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAAI2Zfsmtkft1L0no3Wn4aojvmXW3kCWZ17jgyVaJQo2Jzi7cz8CQQmQlS3HMeaWVuPwd02vfsG5nRJiDv9tfCs8FCYxJwbP_SnYzC3k_SlJJBJttrE7f8FCg--EODkT8SpEcgME5N-At7lDaMURG7MLeRzO_rwKwHSNBpDxTlV8nAKo6-5xCwNCHG6cCuJ1EGT9ks0Zu7yrt5hg2Wg4mIHIHzv9KQURI5eGCtdiBeTny0lNFtJUpGUE_Xbe3ze8EZNqGZ4WjSPBFl09eAUT6RO113lRJ1ytjAeY_6nR3gIqCxVXA2usqBJZYgbLvTSysjpfVCz4iHN1neajiZ-cIo3-GA-tn1a6Z88616iXTdC-hn98Ps3-JxcKb81IArpaenYJpp7c2llzreOE_bx3cIpcSEOWMea7zEh_x1JB_jCjtmOMaFAnoOGqA4SIQ_wZADYCVULdJOLz-",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -31.89595719999999,
                    "lng": 141.5981271
                },
                "viewport": {
                    "northeast": {
                        "lat": -31.89509867010727,
                        "lng": 141.5994562298928
                    },
                    "southwest": {
                        "lat": -31.89779832989272,
            

{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAANN_Fu9lCpDIz1atHz33jxe02rEBASTOFXOaVyT5ia6RAoBCt7JL1retFXjU5hiVHvWyQHyoSzpBN1WsbyQ9LzbDRPn3_34TcgO04_Kn6VVhLk8DAO3HePme7t8TZmaSy1qFbcg0Ft_bARoyFBAle6Xy8k4jVjJoKDvF4s81QZ968yThIL7f4sex89gdcwE_YXihf5-qW6SIMGsCvD7IKkniAxoNCv41s1rl6oSQ8BQ0BiSKStb0WjoUEND_lnoGdeNc7QRG4YwyE130WPOghHwIibhFTtYE021PaVUsyM9VxjNDjlAHo3ZhzIcTXQ7WsLzabpq1tHfCgNenAT9dKYLDsJRfmvzQVGA2bmQA_taHdxvhDVnGjl0GyIzJnPql3rSblFvF6-GHiY2WS1nwfIkSEK88iNsSoDqEFA7F8ceG3WUaFFygBbU6ayvWI9I9rmMyWiSAT84n",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -32.2180258,
                    "lng": -58.13374389999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -32.21668657010728,
                        "lng": -58.13238322010728
                    },
                    "southwest": {
                        "lat": -32.21938622989272,
           

{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAAMa9maf4Y03XKCZzdMPIHWH_BtD2IVzZgz4lEt91GLJr_TWMUYrjQFKeF7gaktpNDpdKUtIg3GKxjttl7ivqnh4m0SyLT0uK9qj5NASpR7J7c_GJ9MfocJyn9O5xv1YLQvEkqePtfLoSvJTw4Wxjt-5i-hY53F5XVeywrcrUcDzjJT-nB_wWyabCeASwGC3w-GtTAWhVLXIh2KvWc9NY8goX9kAh4PBLIs9YHoSxeQAM7f9HFk-ojjGWoz8Yk6tirwT-AcvmPObs9ZK4bWy9cua53e3sdAoKvHOAfCYj8iOtpOK4CbPCgT47yhgbUTobsa3YmyTFQewkvGIfmH1VWENQ2yhj4dRHzJvcUlHVVLr-PNn8a8opeXCnJJv_zMPJQ1OCSH2H4qmQ4Nv_2ODOaggSELYVdxcTDCEvIumpTkp9vEwaFHk8EQil0iiKQvneJ21t6I0dsQNB",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 48.6226593,
                    "lng": 22.9716452
                },
                "viewport": {
                    "northeast": {
                        "lat": 48.62401317989272,
                        "lng": 22.97287707989272
                    },
                    "southwest": {
                        "lat": 48.62131352010728,
                       

{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAAPrpWXDji6erA63-qCDZoUPNjAIHaVlUgWUYIF3HLOMF7P7TCS0utEOpTzj4SMxTNWE3zHgxv9evXKFTFPsAlvFFE4E37a8lozsbOMcGKlTNEKlIp6Kl1RDmvQkCvzm07ZuIaD22y3u870VgbH7cy2MOg2JamXdUrZVQDTjo9SZpo3zzZvYr2E6_UvuxYuKd6HgdCucQEMUKFTZ1P9y2z-bZBbUKwS2KcvqKQ2yebew5b3DXjpWrmb35ARC3LgfJF8ggCth09dfScambTAweET44Ed4LLoPA3nvfLp1uy7dr_qJlYKOAZWM9QFKvG3MdFlXvc1dt81hFq01K32mfXuF2g2mJ-rjeI2I-f3LKJRR4K5l4oB9SShsbMdjz6I4aYQh-ucZS21mnfmwMY3kup-sSEESOwd87TJj7GPMEDV3Ji10aFMbbUlLkxmtPpHr-wqusGXMmHBTw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 45.8168515,
                    "lng": -0.6549743
                },
                "viewport": {
                    "northeast": {
                        "lat": 45.81818467989272,
                        "lng": -0.6536179201072778
                    },
                    "southwest": {
                        "lat": 45.81548502010728,
                     

{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAAHU3eDuBad06M7SGSwdFB3NMKJMhh26t5tBZi-APWTlcSfhUlqXEtVCqDxEsn4u0iZtBE4-x5-WnRU861F-ZR04d_XkLvP-K2FnL_DW-LMODqxkTquVoBxLN955_Kp0nu2OIcoS45x-LOmKPTn-3aKzCb9DMOZTlhnIcmeii5vgsS55NLkWEPgR1Oj-9qK5sMPKOE7XvKzxS-rOEEbKzFpfy1JWcSk7aVUbC9fRZYpraSgzNz3mBnv5Aki_DNkj3OIlefFqk8c1KlXgH67MwTiVQYVT_ZH_cMczZhyLULiQIOYIn4tXZyN6qaCGFjm54kRWKC_eXC-xga0fW2x5D3YAxmMDHWAntBQE3i2pUm9AAG9aVkKWiROI7-BPEFExGRF0a6A1a9UXMLtrIQKN__eMSELstlHNBxvWMKOdqPzU1rGgaFBPXTzDEL6UwzYV87AjAoP9W9NwC",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -27.4748278,
                    "lng": -58.8407361
                },
                "viewport": {
                    "northeast": {
                        "lat": -27.47360607010728,
                        "lng": -58.83940597010729
                    },
                    "southwest": {
                        "lat": -27.47630572989272,
                  

{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAAGUSrpfFI2CZNmzzRlqUJhcG5N5iz3EIPgruCjTQTkVbjxdoEVA-1RORF1QDij_6e1PrR8FIMWhsx5nNpMPd3AJLZGu8IBr6Mq5yPTh2UJDyj0fpNGeuqV754pGFBlkcBo6da2urWXzvDAN00qEqxqKEAc6RW3cIKsS7n6K8T6vLqeRlxNe9WRiYISS1xaGc2tCUBev2HWlPcov_k7etJcyY2oGQ2x_gtvuNtzvA91Hb_44UDTJ8XWHD8UAFU9zC9Bt1XAG_cahEVLbkGZ0J8c2T8xrC3kuZBYD9D4vhnG685jDpOvm-3DwOPC_o2OT62SJEfeaQ4SNTaIJMiTS5hD2zMt5Xlef3QNhZSDTzPIh73GLTxxhBQIEoQo_bGkxHIy5XsIaO-zrtO-WGegs9FZUSEKsgvmQ3o9v5YVfTz_yCdPkaFLZhDkUWqeddYiLZKgX_akJJtXZr",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -25.662399,
                    "lng": -53.80790229999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -25.66101727010728,
                        "lng": -53.80643517010728
                    },
                    "southwest": {
                        "lat": -25.66371692989272,
            

,City ID,Date,Country,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloud Cover,Hotel Name
0,134,2020-04-09 13:48:39,CN,Jingdezhen,29.29,117.21,63.05,61,3.22,0,Fairfield by Marriott Jingdezhen
1,296,2020-04-09 13:49:43,CN,Chaozhou,23.67,116.64,66.20,63,4.47,0,Shantou International Hotel
2,331,2020-04-09 13:49:46,AU,Broken Hill,-31.95,141.43,60.80,59,3.36,0,Broken Hill Outback Resort
3,365,2020-04-09 13:49:50,AR,Concepción del Uruguay,-32.48,-58.24,64.00,64,1.99,0,Costarenas
4,402,2020-04-09 13:49:53,UA,Poroshkovo,48.67,22.75,73.40,17,2.24,0,Polyansky castle
5,416,2020-04-09 13:49:54,FR,Arrondissement de Saintes,45.67,-0.75,75.99,25,4.70,0,Château de Mouillepied
6,451,2020-04-09 13:49:57,PY,General José Eduvigis Díaz,-27.17,-58.42,69.01,55,4.00,0,La Alondra Casa de Huéspedes
7,560,2020-04-09 13:50:06,BR,Realeza,-25.77,-53.53,76.77,28,2.62,0,Hotel Titos


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))